In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.path.abspath('../paper_model')

'/home/hidde/Desktop/git/Vu/Seeing_the_words/bacholar_project/paper_model'

In [8]:
# !python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))


fatal: destination path 'detectron2' already exists and is not an empty directory.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [6]:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.10/index.html


Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.10/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: black
    Found existing installation: black 23.3.0
    Uninstalling black-23.3.0:
      Successfully uninstalled black-23.3.0

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [7]:

import detectron2,cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cpu" # we use a CPU Detectron copy
# create predictor
predictor = DefaultPredictor(cfg)

In [ ]:
file = caffe_root + '/example_img/example_1.jpg'
frame = cv2.imread(file)
image = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
# predict categories
output = predictor(image)


In [ ]:
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(out.get_image()[..., ::-1][..., ::-1])
plt.show()

In [ ]:
matches = zip(output['instances'].get('pred_classes'),output['instances'].get(     'pred_boxes'))

In [ ]:
for l,j in matches:
    # skip all categories which do not correspond to people
    if int(l)!=0 : continue
    # get bounding box for person
    i = [int(k) for k in j]


    # crop the original image using the bounding box
    img = image[i[1]:i[3], i[0]:i[2]] #crop to bb
    plt.imshow(img)
    plt.show()

Section is to get sex and age

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


caffe_root = os.path.abspath('/home/hidde/Desktop/git/Vu/Seeing_the_words/paper_model')
import sys
# sys.path.insert(0, caffe_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [ ]:
caffe_root

In [14]:
mean_filename= caffe_root + '/age_gender_mean.binaryproto'
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

In [ ]:
age_net_pretrained= caffe_root + '/age_net.caffemodel'
age_net_model_file= caffe_root +  '/deploy_age.prototxt'
age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [ ]:
gender_net_pretrained= caffe_root + '/gender_net.caffemodel'
gender_net_model_file= caffe_root + '/deploy_gender.prototxt'
gender_net = caffe.Classifier(gender_net_model_file, gender_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [ ]:
age_list=['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
gender_list=['Male','Female']

In [ ]:
example_image = caffe_root + '/example_img/example_1.jpg'
input_image = caffe.io.load_image(example_image)
_ = plt.imshow(input_image)

In [ ]:
prediction = age_net.predict([input_image])

print ('predicted age:', age_list[prediction[0].argmax()])

In [ ]:
prediction

In [ ]:
prediction = gender_net.predict([input_image])

print ('predicted gender:', gender_list[prediction[0].argmax()])

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cpu" # we use a CPU Detectron copy
# create predictor
predictor = DefaultPredictor(cfg)

In [ ]:
file = caffe_root + '/example_img/example_1.jpg'
frame = cv2.imread(file)
image = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
# predict categories
output = predictor(image)


In [ ]:
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(out.get_image()[..., ::-1][..., ::-1])
plt.show()

In [ ]:
matches = zip(output['instances'].get('pred_classes'),output['instances'].get(     'pred_boxes'))

In [ ]:
for l,j in matches:
    # skip all categories which do not correspond to people
    if int(l)!=0 : continue
    # get bounding box for person
    i = [int(k) for k in j]


    # crop the original image using the bounding box
    img = image[i[1]:i[3], i[0]:i[2]] #crop to bb
    plt.imshow(img)
    plt.show()